In [ ]:
import sys
sys.path.append("../")
from functools import partial
from src.trainer import SQUADTrainer
from src.data import preprocess_function
from datasets import load_dataset
from transformers import (
    TrainingArguments, 
    AutoModelForQuestionAnswering,
    AutoTokenizer
)

In [ ]:
student_model_path = "../pretrained_models/distilbert-base-uncased"
teacher_model_path = "../pretrained_models/distilbert-base-uncased-distilled-squad"
output_dir = "../logs"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(student_model_path)
model = AutoModelForQuestionAnswering.from_pretrained(student_model_path)
teacher_model = AutoModelForQuestionAnswering.from_pretrained(teacher_model_path)

squad = load_dataset("../data/from_hf/squad")

In [ ]:
tokenized_squad = squad.map(
    partial(preprocess_function, tokenizer=tokenizer),
    batched=True,
    remove_columns=squad["train"].column_names
)

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="steps",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    disable_tqdm=True,
    # fp16=True # use with GPU only
)

trainer = SQUADTrainer(
    model=model,
    teacher_model=teacher_model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()